In [73]:
### APO Structure creation ###

from SBLMDCOVDOCK import SBLSettings

settings = SBLSettings.GROMACS_Settings()

import pandas as pd
import os
import subprocess

# collect APO structures 

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"]})



In [74]:
print(settings.pH)

7.4


In [22]:
# Download structures 
# only Chain A

from pdbtools import *



for pdbcode in structures.PDBID:
    output_path = os.path.join(settings.structures_input, pdbcode + ".pdb")
    # !pdb_fetch {pdbcode} | pdb_selchain -A > {output_path.replace(".pdb", "_raw.pdb")}
    !pdb_fetch {pdbcode} | pdb_selchain -A > {output_path}

    # this is only to find dodgy residues DO NOT USE As this removes residues
    pdb4amber_command = [
        "pdb4amber",
        "-i", output_path,
        "-o", output_path.replace(".pdb", "_amber.pdb")
        ]
    subprocess.run(pdb4amber_command)
    print("Done with", pdbcode)


Summary of pdb4amber for: start_structures/APO/6T3D.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
ASP_10
GLN_35
GLY_36
PRO_38
LYS_50
PRO_53
LEU_96
GLN_175
THR_176
LYS_239
ILE_243
LYS_246
MET_265
SER_282
SER_324
VAL_326
GLU_333
LEU_334
ASP_351
ALA_352
TRP_354
HOH_690
HOH_701
HOH_719
HOH_722
HOH_733
-----------Non-standard-resnames
SO4, EDO

---------- Missing heavy atom(s)

GLN_3 misses 3 heavy atom(s)
GLN_4 misses 3 heavy atom(s)
ARG_11 misses 3 heavy atom(s)
TYR_42 misses 1 heavy atom(s)
LYS_48 misses 3 heavy atom(s)
GLN_49 misses 2 heavy atom(s)
GLN_54 misses 3 heavy atom(s)
LYS_81 misses 1 heavy atom(s)
LYS_96 misses 4 heavy atom(s)
ASN_99 misses 2 heavy atom(s)
GLN_117 misses 2 heavy atom(s)
ASP_120 misses 2 heavy atom(s)
GLU_121 misses 4 heavy atom(s)
LYS_123 misses 4 heavy atom(s)
SER_125 misses 1 heavy atom(s)
SER_126 misses 1 heavy atom(s)
ARG_130 m

Done with 6T3D



Summary of pdb4amber for: start_structures/APO/5UL8.pdb


Done with 5UL8



----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
SER_59
GLN_87
ARG_96
TYR_97
VAL_103
TRP_105
SER_106
SER_109
LYS_111
TYR_112
SER_130
ASP_131
GLN_191
LYS_192
VAL_249
VAL_250
VAL_260
LEU_261
ARG_266
ASP_271
ASP_272
-----------Non-standard-resnames
SO4, GOL

---------- Missing heavy atom(s)

None
The alternate coordinates have been discarded.
Only the first occurrence for each atom was kept.

Summary of pdb4amber for: start_structures/APO/1K55.pdb
/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parmed/formats/pdb.py:787: PDBWarning: CONECT record - could not find atoms 2145 and/or 2195 to connect. Line: CONECT 2145 2195                                                                

  warnings.warn(
/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parmed/formats/pdb.py:787: PDBWarning: CONECT record - could not find atoms 2195 and/or 

Done with 1K55


In [23]:
from SBLMDCOVDOCK.md_setuptools import rename_KCX

In [24]:
# Rename KCX from hetatom to atom


for pdbcode in structures.PDBID:
    print(pdbcode)
    input_path = os.path.join(settings.structures_input, pdbcode + ".pdb")
    rename_KCX(input_path)



6T3D
Wrote to start_structures/APO/6T3D_atom.pdb
5UL8
Wrote to start_structures/APO/5UL8_atom.pdb
1K55
changed 1579 HETATM  371  N   KCX A  70    
changed 1580 ANISOU  371  N   KCX A  70    
changed 1581 HETATM  372  CA  KCX A  70    
changed 1582 ANISOU  372  CA  KCX A  70    
changed 1583 HETATM  373  CB  KCX A  70    
changed 1584 ANISOU  373  CB  KCX A  70    
changed 1585 HETATM  374  CG  KCX A  70    
changed 1586 ANISOU  374  CG  KCX A  70    
changed 1587 HETATM  375  CD  KCX A  70    
changed 1588 ANISOU  375  CD  KCX A  70    
changed 1589 HETATM  376  CE  KCX A  70    
changed 1590 ANISOU  376  CE  KCX A  70    
changed 1591 HETATM  377  NZ  KCX A  70    
changed 1592 ANISOU  377  NZ  KCX A  70    
changed 1593 HETATM  378  C   KCX A  70    
changed 1594 ANISOU  378  C   KCX A  70    
changed 1595 HETATM  379  O   KCX A  70    
changed 1596 ANISOU  379  O   KCX A  70    
changed 1597 HETATM  380  CX  KCX A  70    
changed 1598 ANISOU  380  CX  KCX A  70    
changed 1599 HETA

In [25]:
# Create APO - remove water and ligands
for pdbcode in structures.PDBID:
    print(pdbcode)
    input_path = os.path.join(settings.structures_input, pdbcode + ".pdb").replace(".pdb", "_atom.pdb")
    output_path = input_path.replace(pdbcode, "APO_" +pdbcode)
    print(input_path)
    # remove water and ligands
    !pdb_delhetatm {input_path} > {output_path}
    print(output_path)


6T3D
start_structures/APO/6T3D_atom.pdb
start_structures/APO/APO_6T3D_atom.pdb
5UL8
start_structures/APO/5UL8_atom.pdb
start_structures/APO/APO_5UL8_atom.pdb
1K55
start_structures/APO/1K55_atom.pdb
start_structures/APO/APO_1K55_atom.pdb


In [26]:

# Open each PDB file in Chimera to check residue rotamers - make sure that Chimera is in path.
print("cd", os.getcwd())
for pdbcode in structures.PDBID:
    #
    apo_structure = os.path.join(settings.structures_input,"APO_" +pdbcode+"_atom.pdb")
    print("ChimeraX", apo_structure)

cd /Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_2/Project/SBL_MD_CovDock
ChimeraX start_structures/APO/APO_6T3D_atom.pdb
ChimeraX start_structures/APO/APO_5UL8_atom.pdb
ChimeraX start_structures/APO/APO_1K55_atom.pdb


In [19]:
# check PDBs with PDBfixer - remember to rename KCX after
from pdbfixer import PDBFixer
from openmm.app import PDBFile

for pdbcode in structures.PDBID:
    print(pdbcode)
    input_path = os.path.join(settings.structures_input, "APO_" +pdbcode + "_atom.pdb")
    output_path = input_path.replace("_atom.pdb", "_fixed.pdb")
    fixer = PDBFixer(filename=input_path)

    fixer.findMissingResidues() 
    print("Missing Res:", fixer.missingResidues)

    fixer.findMissingAtoms()
    print("Missing Atoms:", fixer.missingAtoms)

    fixer.addMissingAtoms()
    # fixer.removeHeterogens(False) # this removes KCX - goes based on known ligand
    fixer.addMissingHydrogens(settings.pH) 
    print(output_path)
    PDBFile.writeFile(fixer.topology, fixer.positions, open(output_path, 'w')) # change to _fixed

    rename_KCX(output_path) # adds _atom

    pdb4amber_command = [
    "pdb4amber",
    "-i", output_path,
    "-o", output_path.replace(".pdb", "_amber.pdb")
    ]
    subprocess.run(pdb4amber_command)
    print("Done with", pdbcode)

6T3D
Missing Res: {}
Missing Atoms: {<Residue 2 (GLN) of chain 0>: [<Atom 4 (CD) of chain 0 residue 0 (GLN)>, <Atom 5 (OE1) of chain 0 residue 0 (GLN)>, <Atom 6 (NE2) of chain 0 residue 0 (GLN)>], <Residue 3 (GLN) of chain 0>: [<Atom 4 (CD) of chain 0 residue 0 (GLN)>, <Atom 5 (OE1) of chain 0 residue 0 (GLN)>, <Atom 6 (NE2) of chain 0 residue 0 (GLN)>], <Residue 10 (ARG) of chain 0>: [<Atom 6 (CZ) of chain 0 residue 0 (ARG)>, <Atom 7 (NH1) of chain 0 residue 0 (ARG)>, <Atom 8 (NH2) of chain 0 residue 0 (ARG)>], <Residue 41 (TYR) of chain 0>: [<Atom 7 (OH) of chain 0 residue 0 (TYR)>], <Residue 47 (LYS) of chain 0>: [<Atom 4 (CD) of chain 0 residue 0 (LYS)>, <Atom 5 (CE) of chain 0 residue 0 (LYS)>, <Atom 6 (NZ) of chain 0 residue 0 (LYS)>], <Residue 48 (GLN) of chain 0>: [<Atom 5 (OE1) of chain 0 residue 0 (GLN)>, <Atom 6 (NE2) of chain 0 residue 0 (GLN)>], <Residue 53 (GLN) of chain 0>: [<Atom 4 (CD) of chain 0 residue 0 (GLN)>, <Atom 5 (OE1) of chain 0 residue 0 (GLN)>, <Atom 6 (NE2


Summary of pdb4amber for: start_structures/APO/APO_6T3D_fixed.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None


Done with 6T3D
5UL8
Missing Res: {}
Missing Atoms: {}
start_structures/APO/APO_5UL8_fixed.pdb
Wrote to start_structures/APO/APO_5UL8_fixed_atom.pdb



Summary of pdb4amber for: start_structures/APO/APO_5UL8_fixed.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None


Done with 5UL8
1K55
Missing Res: {(0, 245): ['GLY']}
Missing Atoms: {}
start_structures/APO/APO_1K55_fixed.pdb
changed 729 HETATM  728  N   KCX A  50    
changed 730 HETATM  729  CA  KCX A  50    
changed 731 HETATM  730  CB  KCX A  50    
changed 732 HETATM  731  CG  KCX A  50    
changed 733 HETATM  732  CD  KCX A  50    
changed 734 HETATM  733  CE  KCX A  50    
changed 735 HETATM  734  NZ  KCX A  50    
changed 736 HETATM  735  C   KCX A  50    
changed 737 HETATM  736  O   KCX A  50    
changed 738 HETATM  737  CX  KCX A  50    
changed 739 HETATM  738  OQ1 KCX A  50    
changed 740 HETATM  739  OQ2 KCX A  50    
Wrote to start_structures/APO/APO_1K55_fixed_atom.pdb



Summary of pdb4amber for: start_structures/APO/APO_1K55_fixed.pdb


Done with 1K55



----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames
KCX

---------- Gaps (Renumbered Residues!)
gap of 3.039053 A between PHE 49 and ILE 51

---------- Missing heavy atom(s)

None


In [27]:
# Open each PDB file in Chimera to check residue rotamers - make sure that Chimera is in path.
print("cd", os.getcwd())
for pdbcode in structures.PDBID:
    #
    apo_structure = os.path.join(settings.structures_input,"APO_" +pdbcode+"_fixed_atom.pdb")
    print("ChimeraX", apo_structure)
    print(apo_structure.replace("_fixed_atom.pdb", "_chimera.pdb"))

cd /Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_2/Project/SBL_MD_CovDock
ChimeraX start_structures/APO/APO_6T3D_fixed_atom.pdb
start_structures/APO/APO_6T3D_chimera.pdb
ChimeraX start_structures/APO/APO_5UL8_fixed_atom.pdb
start_structures/APO/APO_5UL8_chimera.pdb
ChimeraX start_structures/APO/APO_1K55_fixed_atom.pdb
start_structures/APO/APO_1K55_chimera.pdb


In [38]:
### Upload to Molprobity

suffix = "_chimera.pdb"

In [41]:
# Check missing atoms and residues in the structures

for pdbcode in structures.PDBID:
    print(pdbcode)
    input_path = os.path.join(settings.structures_input, "APO_"+ pdbcode + suffix)
    print(input_path)
    fixer = PDBFixer(filename=input_path)

    fixer.findMissingResidues() 
    print("Missing Res:", fixer.missingResidues)
    fixer.findMissingAtoms()
    print("Missing Atoms:", fixer.missingAtoms)

    # this is only to find dodgy residues - removes residues WRONG!!!
    pdb4amber_command = [
        "pdb4amber",
        "-i", input_path,
        "-o", input_path.replace(".pdb", "_amber.pdb")
        ]
    subprocess.run(pdb4amber_command)
    print("Done with", pdbcode)

    !propka3 {input_path} -o 7.4 


6T3D
start_structures/APO/APO_6T3D_chimera.pdb
Missing Res: {}
Missing Atoms: {}



Summary of pdb4amber for: start_structures/APO/APO_6T3D_chimera.pdb


Done with 6T3D


/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parmed/formats/pdb.py:827: PDBWarning: Could not find link atoms AtomLookupKey(name='NE2', number=None, residue_name='HIS', residue_number='13', chain='A', insertion_code='', segment_id='', alternate_location=None) and AtomLookupKey(name='ZN', number=None, residue_name='ZN', residue_number='402', chain='A', insertion_code='', segment_id='', alternate_location=None)
  warnings.warn(f'Could not find link atoms {key1} and {key2}', PDBWarning)

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
ASP_10
GLN_35
GLY_36
PRO_38
LYS_50
PRO_53
LEU_96
GLN_175
THR_176
LYS_239
ILE_243
LYS_246
MET_265
SER_282
SER_324
VAL_326
GLU_333
LEU_334
ASP_351
ALA_352
TRP_354
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None
The alternate coordinates have been discarded.
Only the first occurrence for each at

5UL8
start_structures/APO/APO_5UL8_chimera.pdb
Missing Res: {}
Missing Atoms: {}


...

Done with 5UL8



----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
SER_59
GLN_87
ARG_96
TYR_97
VAL_103
TRP_105
SER_106
SER_109
LYS_111
TYR_112
SER_130
ASP_131
GLN_191
LYS_192
VAL_249
VAL_250
VAL_260
LEU_261
ARG_266
ASP_271
ASP_272
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None
The alternate coordinates have been discarded.
Only the first occurrence for each atom was kept.


1K55
start_structures/APO/APO_1K55_chimera.pdb
Missing Res: {(0, 245): ['GLY']}
Missing Atoms: {}



Summary of pdb4amber for: start_structures/APO/APO_1K55_chimera.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
LYS_91
MET_99
SER_147
VAL_211
-----------Non-standard-resnames
KCX

---------- Gaps (Renumbered Residues!)
gap of 3.039053 A between PHE 49 and ILE 51

---------- Missing heavy atom(s)

None
The alternate coordinates have been discarded.
Only the first occurrence for each atom was kept.


Done with 1K55


In [ ]:
# Check missing atoms and residues in the structures

for pdbcode in structures.PDBID:
    print(pdbcode)
    input_path = os.path.join(settings.structures_input, "APO_"+ pdbcode + suffix)
    print(input_path)
    fixer = PDBFixer(filename=input_path)

    fixer.findMissingResidues() 
    print("Missing Res:", fixer.missingResidues)
    fixer.findMissingAtoms()
    print("Missing Atoms:", fixer.missingAtoms)

    # this is only to find dodgy residues 
    pdb4amber_command = [
        "pdb4amber",
        "-i", input_path,
        "-o", input_path.replace(".pdb", "_amber.pdb")
        ]
    subprocess.run(pdb4amber_command)
    print("Done with", pdbcode)

    !propka3 {input_path} -o 7.4 


6T3D
start_structures/APO/APO_6T3D_chimera.pdb
Missing Res: {}
Missing Atoms: {}



Summary of pdb4amber for: start_structures/APO/APO_6T3D_chimera.pdb


Done with 6T3D


/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parmed/formats/pdb.py:827: PDBWarning: Could not find link atoms AtomLookupKey(name='NE2', number=None, residue_name='HIS', residue_number='13', chain='A', insertion_code='', segment_id='', alternate_location=None) and AtomLookupKey(name='ZN', number=None, residue_name='ZN', residue_number='402', chain='A', insertion_code='', segment_id='', alternate_location=None)
  warnings.warn(f'Could not find link atoms {key1} and {key2}', PDBWarning)

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
ASP_10
GLN_35
GLY_36
PRO_38
LYS_50
PRO_53
LEU_96
GLN_175
THR_176
LYS_239
ILE_243
LYS_246
MET_265
SER_282
SER_324
VAL_326
GLU_333
LEU_334
ASP_351
ALA_352
TRP_354
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None
The alternate coordinates have been discarded.
Only the first occurrence for each at

5UL8
start_structures/APO/APO_5UL8_chimera.pdb
Missing Res: {}
Missing Atoms: {}


...

Done with 5UL8



----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
SER_59
GLN_87
ARG_96
TYR_97
VAL_103
TRP_105
SER_106
SER_109
LYS_111
TYR_112
SER_130
ASP_131
GLN_191
LYS_192
VAL_249
VAL_250
VAL_260
LEU_261
ARG_266
ASP_271
ASP_272
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None
The alternate coordinates have been discarded.
Only the first occurrence for each atom was kept.


1K55
start_structures/APO/APO_1K55_chimera.pdb
Missing Res: {(0, 245): ['GLY']}
Missing Atoms: {}



Summary of pdb4amber for: start_structures/APO/APO_1K55_chimera.pdb

----------Chains
The following (original) chains have been found:
A

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
LYS_91
MET_99
SER_147
VAL_211
-----------Non-standard-resnames
KCX

---------- Gaps (Renumbered Residues!)
gap of 3.039053 A between PHE 49 and ILE 51

---------- Missing heavy atom(s)

None
The alternate coordinates have been discarded.
Only the first occurrence for each atom was kept.


Done with 1K55


# Upload to Molprobity

6T3D
Flip? 	Chain 	Res# 	Alt 	Res ID 	Orig 	Flip 	Flip-Orig 	Code 	Explanation
	A 	52 		GLN 	-3.2 	-2 	1.2 	CLS-FL 	Both orientations clash but flip was preferred.
	A 	120 		GLN 	-7.9 	0.24 	8.14 	CLS-FL 	Both orientations clash but flip was preferred.
	A 	136 		GLN 	-9.6 	-7.3 	2.3 	CLS-FL 	Both orientations clash but flip was preferred.

5UL8 
Reduce didn't flip any groups while adding hydrogens to your file. This may indicate that all of the Asn's, Gln's, and His's in your structure are oriented correctly. (Show all Asn/Gln/His)

1K55
Flip? 	Chain 	Res# 	Alt 	Res ID 	Orig 	Flip 	Flip-Orig 	Code 	Explanation
	A 	73 		ASN 	-1.8 	-0.1 	1.7 	FLIP 	Some evidence recommending flip.
	A 	165 		ASN 	-2.1 	0.41 	2.51 	FLIP 	Clear evidence for flip.


In [52]:
# Use Molprobity structures for KCX - no need to protonate
suffix = "_chimeraFH.pdb"

In [70]:
for pdbcode in structures.PDBID:
    print(pdbcode)
    input_path = os.path.join(settings.structures_input, "APO_"+ pdbcode + suffix)
    
    !pdb4amber -i {input_path} -o {input_path.replace(".pdb", "_amber.pdb")}

6T3D

Summary of pdb4amber for: start_structures/APO/APO_6T3D_chimeraFH.pdb
/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parmed/formats/pdb.py:787: PDBWarning: CONECT record - could not find atoms 5602 and/or 5611 to connect. Line: CONECT 5602 5611 5603 5585

  warnings.warn(
/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parmed/formats/pdb.py:787: PDBWarning: CONECT record - could not find atoms 5611 and/or 5602 to connect. Line: CONECT 5611 5602

  warnings.warn(
/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parmed/formats/pdb.py:787: PDBWarning: CONECT record - could not find atoms 5612 and/or 5603 to connect. Line: CONECT 5612 5603

  warnings.warn(
/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parmed/formats/pdb.py:787: PDBWarning: CONECT record - could not find atoms 5613 and/or 5606 to connect. Line: CONECT 5613 5606

  warnings.warn(
/Users/alexi/miniconda/envs/RIN_test/lib/python3.8/site-packages/parme

In [53]:
# Create KCX pdb and find residue number - only need to find one.
# Use Molprobity structure
for pdbcode in structures.PDBID:
    print(pdbcode)
    output_path = os.path.join(settings.structures_input, "APO_"+ pdbcode + suffix)
    !pdb_selresname -KCX {output_path} | pdb_selchain -A > {output_path.replace(".pdb", "_KCX.pdb")}
    print(output_path.replace(".pdb", "_KCX.pdb"))
    with open(output_path.replace(".pdb", "_KCX.pdb"), "r") as f:
        lines = f.readlines()
        for line in lines:
            split = line.split("  ")
            # print(split)
            # split.remove("")
            # print(split)
            if split[0] == "ATOM":
                print(split[5])
                kcx_residue = int(split[5])
                break
    
# perform parameterisation using antechamber 


6T3D


start_structures/APO/APO_6T3D_chimeraFH_KCX.pdb
5UL8
start_structures/APO/APO_5UL8_chimeraFH_KCX.pdb
1K55
start_structures/APO/APO_1K55_chimeraFH_KCX.pdb
70


In [54]:
# Grab residues before and after kcx_residue including KCX
# 
for pdbcode in structures.PDBID:
    print(pdbcode)
    input_path = os.path.join(settings.structures_input, "APO_"+ pdbcode + suffix)
    # select residues before and after kcx_residue
    start = kcx_residue - 1
    end = kcx_residue + 1
    !pdb_selres -{start},{kcx_residue},{end} {input_path} > {input_path.replace(".pdb", "_KCX-1+1.pdb")}
    # !pdb_selres -f {input_path}  -{before_kcx}:{after_kcx} > {input_path.replace(".pdb", "_KCX-1+1.pdb")}

6T3D
5UL8
1K55


In [60]:
os.getcwd()
os.listdir(settings.structures_input)

['APO_1K55_prot74.pqr',
 '#posre.itp.1#',
 'APO_6YPD_prot74.pdb',
 '5UL8_nonprot.pdb',
 '5UL8_renum.txt',
 '1K55_atom_KCX-1+1.pdb',
 'APO_6T3D_chimera_amber.pdb',
 'APO_5UL8_prot74.log',
 '#1k55.top.4#',
 'APO_1K55_fixed_amber_sslink',
 '1K55_atom.pdb',
 '#posre.itp.2#',
 'APO_6T3D_chimera_KCX-1+1.pdb',
 '1K55_renum.txt',
 '5UL8_atom.pdb',
 'APO_1K55_fixed_atom_prot74.pdb',
 'APO_6T3D_fixed_atom_prot74.pdb',
 '5UL8_atom_KCX-1+1.pdb',
 '#1K55.gro.3#',
 'posre.itp',
 'APO_5UL8_chimera_KCX-1+1.pdb',
 '#1k55.top.7#',
 '.DS_Store',
 'APO_6T3D_atom.pdb',
 '5UL8_fixed.pdb',
 'APO_5UL8_fixed_amber_nonprot.pdb',
 '5UL8_amber_sslink',
 'APO_5UL8_chimeraFH.pdb',
 'APO_6T3D_chimeraFH_KCX.pdb',
 'APO_6T3D_chimeraFH.pdb',
 'KCXparams_GMX.gro',
 '1K55_amber_renum.txt',
 'KCXparams_GMX.itp',
 'APO_5UL8_chimeraFH_KCX-1+1.pdb',
 'APO_5UL8_chimera_amber_renum.txt',
 'APO_6T3D_prot74.pqr',
 '6T3D_BKCX.pdb',
 'APO_6T3D_chimera_KCX.pdb',
 '1K55_fixed.pdb',
 'APO_1K55_chimeraFH.pdb',
 'APO_1K55_chimera_amber

In [66]:
# T
import pdb2pqr

import subprocess




for pdbcode in structures.PDBID:
    input_path = os.path.join(settings.structures_input, "APO_"+ pdbcode + "_chimera.pdb")

    print(input_path)
    command = [
        "pdb2pqr",
        "--ff=AMBER",
        "--with-ph=7.4",
        "--titration-state-method=propka",
        input_path,
        "-ffout=AMBER",
        input_path.replace(".pdb", "_prot74.pqr"),
        "--pdb-output",
        input_path.replace(".pdb", "_prot74.pdb"),
    ]
    try:
        subprocess.run(command, check=True)
    except:
        pass



start_structures/APO/APO_6T3D_chimera.pdb


INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: start_structures/APO/APO_6T3D_chimera.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 358 residues and 5298 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 1 missing atoms in biomolecule.
INFO:Added atom OXT to residue GLN A 361 at coordinates -15.299, -12.280, 6.697
INFO:Updating disulfide bridges.
INFO:Debumping biomolecule.
INFO:Assigning titration states with PROPKA.
INFO:
propka3.5

start_structures/APO/APO_5UL8_chimera.pdb


INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: start_structures/APO/APO_5UL8_chimera.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 271 residues and 4031 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:This biomolecule is clean.  No repair needed.
INFO:Updating disulfide bridges.
INFO:Debumping biomolecule.
INFO:Assigning titration states with PROPKA.
INFO:
propka3.5.0                                                                                  202

start_structures/APO/APO_1K55_chimera.pdb


INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: start_structures/APO/APO_1K55_chimera.pdb
INFO:Setting up molecule.
INFO:Created biomolecule object with 245 residues and 1938 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 1 missing atoms in biomolecule.
INFO:Added atom OXT to residue GLY A 265 at coordinates 4.642, 6.704, 60.948
INFO:Updating disulfide bridges.
INFO:Debumping biomolecule.
INFO:Assigning titration states with PROPKA.
INFO:
propka3.5.0 

In [69]:
from SBLMDCOVDOCK.md_setuptools import parse_propka_output

for pdbcode in structures.PDBID:
    print(pdbcode)
    
    cleaned_path = os.path.join(settings.structures_input, "APO_"+ pdbcode + "_fixed.pdb").replace(".pdb", "_atom.pdb")
    cleaned_path =  ("APO_"+ pdbcode + "_fixed.pdb").replace(".pdb", "_atom.pdb")


    protonation_states = parse_propka_output(cleaned_path.replace(".pdb", ".pka"), pH=7.4)
    protonation_states = pd.DataFrame.from_dict(protonation_states, orient="index")
    print(protonation_states)


6T3D
            0
(ASP, 7)    0
(ASP, 44)   0
(ASP, 73)   0
(ASP, 84)   0
(ASP, 120)  0
...        ..
(ARG, 229)  1
(ARG, 255)  1
(ARG, 293)  1
(ARG, 306)  1
(ARG, 346)  1

[66 rows x 1 columns]
5UL8
            0
(ASP, 17)   0
(ASP, 28)   0
(GLU, 9)    0
(GLU, 15)   0
(LYS, 13)   1
(ASP, 34)   0
(ASP, 73)   0
(ASP, 99)   0
(ASP, 105)  0
(ASP, 118)  0
(ASP, 121)  0
(ASP, 151)  0
(ASP, 170)  0
(ASP, 175)  0
(ASP, 188)  1
(GLU, 5)    0
(GLU, 6)    0
(GLU, 52)   0
(GLU, 63)   0
(GLU, 83)   0
(GLU, 108)  0
(GLU, 110)  0
(GLU, 130)  0
(HIS, 161)  0
(LYS, 15)   1
(LYS, 41)   1
(LYS, 53)   1
(LYS, 82)   1
(LYS, 134)  1
(LYS, 154)  1
(LYS, 176)  0
(ARG, 3)    1
(ARG, 7)    1
(ARG, 25)   1
(ARG, 38)   1
(ARG, 95)   1
(ARG, 103)  1
(ARG, 106)  1
(ARG, 120)  1
(ARG, 126)  1
(ARG, 146)  1
(ARG, 162)  1
(ARG, 164)  1
(ASP, 18)   0
(ASP, 19)   0
(GLU, 23)   0
(GLU, 35)   0
(HIS, 21)   0
(LYS, 17)   1
(LYS, 20)   1
(ARG, 13)   1
(ARG, 31)   1
1K55
            0
(ASP, 35)   0
(ASP, 73)   0
(ASP, 85) 

In [159]:
# Download and clean KCX
resname  = "KCX"
link = f"https://www.ebi.ac.uk/pdbe/static/files/pdbechem_v2/{resname}.cif"
import requests
kcx_dir = os.path.join(settings.structures_input, resname)
print(kcx_dir)
# download to KCX dir

# Download the CIF file and save it to the KCX directory
cif_filename = os.path.join(kcx_dir, f"{resname}.cif")
response = requests.get(link)
if response.status_code == 200:
    with open(cif_filename, 'wb') as file:
        file.write(response.content)
    print(f"CIF file downloaded and saved to {cif_filename}")
else:
    print(f"Failed to download the CIF file. Status code: {response.status_code}")


# #remove HQ2 entries
# with open(cif_filename, "r") as f:
#     lines = f.readlines()

# for idx,line in enumerate(lines):

#     if "HQ2" in line:
#         # print(line)
#         lines[idx] = ""
#         print("changed", idx, line[:30])



# # #Update formula H14 -> H13
# #     if "formula" in line:
# #         # print(line)
# #         lines[idx] = line.replace("H14", "H13")
# #         print("changed", idx, line[:30])

# # Change formal charge from 0 to -1

#     if "formal_charge" in line:
#         # print(line)
#         lines[idx] = line.replace("0", "-1")
#         print("changed", idx, line[:35])
# # Change bond order from 1 to 2 for OQ2

#     if ("OQ2" in line) and (" CX" in line):
#         # print(line)
#         lines[idx] = line.replace("SING", "DOUB")
#         print("changed", idx, line[:30])

# # write CIF file
# with open(cif_filename.replace(".cif","_COO.cif"), "w+") as f:
#     f.writelines(lines)

# print("written to", cif_filename.replace(".cif","_COO.cif"))

start_structures/APO/KCX
CIF file downloaded and saved to start_structures/APO/KCX/KCX.cif


In [166]:
# load into antechamber

cif_filename = os.path.join(kcx_dir, f"{resname}.cif")
# cif_filename = cif_filename.replace(".cif", "_COO.cif")

antechamber_comand = [
    'antechamber',
    '-i', cif_filename,
    '-fi', 'ccif',
    '-o', cif_filename.replace(".cif", ".ac"),
    '-fo', 'ac',
    '-bk', 'KCX',
    '-nc', '0',
    '-at', 'amber',
]

subprocess.run(antechamber_comand)


Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.





/Users/alexi/miniconda/envs/RIN_test/bin/wrapped_progs/antechamber: Non Fatal Error!
Residue (KCX) has a type of LINKING.
    This linking monomer will likely become part of a polymer
    and thus probably requires special handling beyond antechamber.
Ignore this error only if you know what you are doing.



CompletedProcess(args=['antechamber', '-i', 'start_structures/APO/KCX/KCX.cif', '-fi', 'ccif', '-o', 'start_structures/APO/KCX/KCX.ac', '-fo', 'ac', '-bk', 'KCX', '-nc', '-1', '-at', 'amber', '-j', '5'], returncode=0)

In [167]:

# only in the case of manual deprotonation
mainchain_COO_file = """

"""


mainchain_file = """
HEAD_NAME N
TAIL_NAME C
MAIN_CHAIN CA
MAIN_CHAIN CB
MAIN_CHAIN CG
MAIN_CHAIN CD
MAIN_CHAIN CE
MAIN_CHAIN NZ
MAIN_CHAIN CX
MAIN_CHAIN OQ1
MAIN_CHAIN OQ2
OMIT_NAME HN2
OMIT_NAME OXT
OMIT_NAME HXT
OMIT_NAME HQ2
PRE_HEAD_TYPE C
POST_TAIL_TYPE N
CHARGE -1.0
"""

mc_file = os.path.join(kcx_dir, f"{resname}.mc")

with open(mc_file, "w+") as f:
    f.write(mainchain_file)

print(cif_filename)
prepgen_command = [
    "prepgen",
    "-i", resname + ".ac",
    "-o", resname + ".prepin",
    "-m", resname +".mc",
    "-rn", resname,
]
# THis only works when you select cwd
subprocess.run(prepgen_command, check=True, cwd=kcx_dir)

start_structures/APO/KCX/KCX.cif

PRE_HEAD_TYPE is     C
POST_TAIL_TYPE is     N
Net charge of truncated molecule is    -1.00
HEAD_ATOM      1    N
TAIL_ATOM      8    C
MAIN_CHAIN     1    1    N
MAIN_CHAIN     2    2   CA
MAIN_CHAIN     3    3   CB
MAIN_CHAIN     4    4   CG
MAIN_CHAIN     5    5   CD
MAIN_CHAIN     6    6   CE
MAIN_CHAIN     7    7   NZ
MAIN_CHAIN     8   10   CX
MAIN_CHAIN     9   12  OQ1
MAIN_CHAIN    10   13  OQ2
MAIN_CHAIN    11    8    C
OMIT_ATOM      1   15  HN2
OMIT_ATOM      2   11  OXT
OMIT_ATOM      3   26  HXT
OMIT_ATOM      4   27  HQ2
Number of mainchain atoms (including head and tail atom):    11
Number of omited atoms:     4
Info: There is a bond linking a non-head and non-tail residue atom (OQ2) and an omitted atom (HQ2).
      You need to specifically add this bond in LEaP using the command 'bond <atom1> <atom2> [order]'
      to link OQ2 to an atom in another residue (similar to disulfide bonds)!

Total postive charge is 0.0
Total negtive charge i

CompletedProcess(args=['prepgen', '-i', 'KCX.ac', '-o', 'KCX.prepin', '-m', 'KCX.mc', '-rn', 'KCX'], returncode=0)

In [170]:
ff19SB_parm = "parm19.dat"
ff_dat_path = os.environ.get('AMBERHOME')+"/dat/leap/parm/"

parmchk_command = [
    "parmchk2",
    '-i', cif_filename.replace(".cif", ".prepin"),
    '-f', 'prepi',
    '-o', cif_filename.replace(resname+".", "frcmod.").replace("cif", resname),
    # "-p", os.path.join(ff_dat_path, ff19SB_parm)
]

# print(" ".join(parmchk_command))
subprocess.run(parmchk_command,check=True)
# subprocess.run("echo $AMBERHOME")
# print(" ".join(parmchk_command))

CompletedProcess(args=['parmchk2', '-i', 'start_structures/APO/KCX/KCX.prepin', '-f', 'prepi', '-o', 'start_structures/APO/KCX/frcmod.KCX'], returncode=0)

In [163]:
KCX_note = """; KCX residue topology MODIFIED by Alexi on 31/07/2023 """

KCX_itp_atoms = """

[ atoms ]

;   nr  type  resi  res  atom  cgnr     charge      mass       ; qtot   bond_type

     1   N     1   KCX     N    1    -0.903800     14.01000 ; qtot -0.904

     2   CT     1   KCX    CA    2     0.127500     12.01000 ; qtot -0.776

     3   CT     1   KCX    CB    3    -0.117400     12.01000 ; qtot -0.894

     4   CT     1   KCX    CG    4    -0.079400     12.01000 ; qtot -0.973

     5   cT     1   KCX    CD    5    -0.092400     12.01000 ; qtot -1.065

     6   CT     1   KCX    CE    6     0.090000     12.01000 ; qtot -0.975

     7    N     1   KCX    NZ    7    -0.601900     14.01000 ; qtot -1.577 

     8    C     1   KCX     C    8     0.641100     12.01000 ; qtot -0.936

     9    O     1   KCX     O    9    -0.559000     16.00000 ; qtot -1.495

    10    C     1   KCX    CX   10     0.989600     12.01000 ; qtot -0.506

    11   OH     1   KCX   OXT   11    -0.607100     16.00000 ; qtot -1.113

    12   O2     1   KCX   OQ1   12    -0.843300     16.00000 ; qtot -1.956

    13   O2     1   KCX   OQ2   13    -0.843300     16.00000 ; qtot -2.799

    14   H     1   KCX     H   14     0.363800      1.00800 ; qtot -2.436

    15   H     1   KCX   HN2   15     0.363800      1.00800 ; qtot -2.072

    16   H1     1   KCX    HA   16     0.091700      1.00800 ; qtot -1.980

    17   HC     1   KCX   HB2   17     0.055200      1.00800 ; qtot -1.925

    18   HC     1   KCX   HB3   18     0.055200      1.00800 ; qtot -1.870

    19   HC     1   KCX   HG2   19     0.037700      1.00800 ; qtot -1.832

    20   HC     1   KCX   HG3   20     0.037700      1.00800 ; qtot -1.794

    21   HC     1   KCX   HD2   21     0.036200      1.00800 ; qtot -1.758

    22   HC     1   KCX   HD3   22     0.036200      1.00800 ; qtot -1.722

    23   H1     1   KCX   HE2   23     0.022200      1.00800 ; qtot -1.700

    24   H1     1   KCX   HE3   24     0.022200      1.00800 ; qtot -1.678

    25   H     1   KCX    HZ   25     0.241500      1.00800 ; qtot -1.436

    26   HO     1   KCX   HXT   26     0.436000      1.00800 ; qtot -1.000

"""
KCX_itp_partial_charges = [-0.9038,0.1275,-0.1174,-0.0794,-0.0924,0.09,-0.6019,0.6411,-0.559,0.9896,-0.6071,-0.8433, -0.8433,0.3638,0.3638,0.0917,0.0552,0.0552,0.0377,0.0377,0.0362,0.0362,0.0222,0.0222,0.2415,0.436]
print(sum(KCX_itp_partial_charges))
print(sum(KCX_itp_partial_charges) -- 1.0)


-1.0000000000000002
-2.220446049250313e-16


In [164]:
# with open("KCXparams_GMX.itp", "r") as input_file:
#     atomtypes_section = False
#     for line in input_file:
#         line = line.strip()
#         if "[ atomtypes ]" in line:
#             atomtypes_section = True
#             continue
#         if atomtypes_section and line and not line.startswith(";"):
#             columns = line.split()
#             print(f"{columns[0]:<8s}{columns[2]:<10s}{columns[3]:<10s}A   {columns[5]:<10s}{columns[6]}")
#         if atomtypes_section and (not line or line.startswith(";")):
#             break

/Users/alexi/miniconda/envs/RIN_test


In [ ]:
# ### Topology found in https://mailman-1.sys.kth.se/pipermail/gromacs.org_gmx-users/2016-January/103280.html
# not using this, writing my own

In [ ]:
# ADD KCX to topology
### https://manual.gromacs.org/current/how-to/topology.html#
break # this will overide the changes TOOD, make changes a function
# Copy amber99 topology to working directory -
### Modifying a force field is best done by making a full copy of the installed forcefield directory and residuetypes.dat into your local working directory:
#Then, modify those local copies as above. pdb2gmx will then find both the original and modified version and you can choose the modified version interactively from the list, or if you use the pdb2gmx -ff option the local version will override the system version.
gmxtop = "/usr/local/gromacs/share/gromacs/top"
command = f"cp -r {gmxtop}/residuetypes.dat {gmxtop}/amber99sb-ildn.ff ."

os.system(command)



In [ ]:
#use BioBB Ligand Parameterisation to generate the topology and parameters for the ligand
# manually adapt them to rtp topology file:
KCX_rtp = """
; Added by Alexi 31/07/23
[ KCX ]
 [ atoms ]
     N    N           -0.34790     1
     H    H            0.27470     2
    CA    CT          -0.24000     3
    HA    H1           0.14260     4
    CB    CT          -0.00940     5
   HB1    HC           0.03620     6
   HB2    HC           0.03620     7
    CG    CT           0.01870     8
   HG1    HC           0.01030     9
   HG2    HC           0.01030    10
    CD    CT          -0.04790    11
   HD1    HC           0.06210    12
   HD2    HC           0.06210    13
    CE    CT          -0.01430    14
   HE1    HP           0.11350    15
   HE2    HP           0.11350    16
    NZ    N           -0.38540    17
   HZ1    H            0.34000    18
     C    C            0.73410    19
     O    O           -0.58940    20
    CX    C            0.73410    21
   OQ1    O2          -0.81880    22 ;taken from GLU
   OQ2    O2          -0.81880    23 ;taken from GLU

 [ bonds ]
     N     H
     N    CA
    CA    HA
    CA    CB
    CA     C
    CB   HB1
    CB   HB2
    CB    CG
    CG   HG1
    CG   HG2
    CG    CD
    CD   HD1
    CD   HD2
    CD    CE
    CE   HE1
    CE   HE2
    CE    NZ
    NZ   HZ1
     C     O
    -C     N
    NZ    CX
    CX    OQ1
    CX    OQ2

 [ impropers ]
    -C    CA     N     H
    CA    +N     C     O
    N     O2     C    O2 ;Adapted from GLU - listed as X  O2  C  O2 in ffbonded.itp
    C     CT     N    H ;listed as X  CT  N  H in ffbonded.itp
    """
# Check partial charges
KCX_charges = [-0.3479,
 0.2747,
 -0.24,
 0.1426,
 -0.0094,
 0.0362,
 0.0362,
 0.0187,
 0.0103,
 0.0103,
 -0.0479,
 0.0621,
 0.0621,
 -0.0143,
 0.1135,
 0.1135,
 -0.3854,
 0.34,
 0.7341,
 -0.5894,
 0.7341,
 -0.8188,
 -0.8188]



KCX_charges = [-0.3479,
 0.2747,
 -0.24,
 0.1426,
 -0.0094,
 0.0362,
 0.0362,
 0.0187,
 0.0103,
 0.0103,
 -0.0479,
 0.0621,
 0.0621,
 -0.0143,
 0.1135,
 0.1135,
 -0.3854,
 0.34,
 0.7341,
 -0.5894,
 0.7342,
 -1.0271,
 -1.0271]

print(sum(KCX_charges))
print(sum(KCX_charges)--1)

import numpy as np
KCX_charges = np.array(KCX_charges)
KCX_charges = KCX_charges*(-1/KCX_charges.sum())
print(KCX_charges.round(5))

In [ ]:
KCX_hdb = """
; Added by Alexi 31/07/2023
KCX	7
1	1	H	N	-C	CA	
1	5	HA	CA	N	CB	C	
2	6	HB	CB	CA	CG	
2	6	HG	CG	CB	CD	
2	6	HD	CD	CG	CE	
2	6	HE	CE	CD	NZ	
1	4	HZ	NZ	CE	CD	;changed from 3->1
"""

In [ ]:
for pdbcode in structures.PDBID:
    input_path = os.path.join(settings.structures_input, "APO_" + pdbcode + ".pdb")

    output_path = os.path.join(settings.structures_output, "APO_" + pdbcode + ".gro")
    topo_path = os.path.join(settings.structures_output, "APO_" + pdbcode + ".top")
    print(input_path)
    print(output_path)
    print(topo_path)
    command = ["gmx", "pdb2gmx", 
                    "-f", input_path, 
                    "-o", output_path, 
                    "-p", topo_path, "-water", "tip3p", 
                    "-ff", "amber99sb-ildn",
                    "-ignh",
                    "-inter"]
    # subprocess.run(command, check=True)
### THIS has to be done manually, because the program has interactive input
# need to add KCX to the topology file
# HIS set to HISE (HIE) when done non-interactively - is this correct?
# Check with _prot74.pdb - protons not recognised
# 6T3D only works with this method - everything set to default TODO Check this as we still set -ignh
#
# gmx pdb2gmx -f start_structures/APO/APO_1K55.pdb -o prod_structures/APO/APO_1K55.gro -p prod_structures/APO/APO_1K55.top -water tip3p -ff amber99sb-ildn -ignh -inter
# gmx pdb2gmx -f start_structures/APO/APO_5UL8.pdb -o prod_structures/APO/APO_5UL8.gro -p prod_structures/APO/APO_5UL8.top -water tip3p -ff amber99sb-ildn -ignh -inter
# gmx pdb2gmx -f start_structures/APO/APO_6T3D_prot74.pdb -o prod_structures/APO/APO_6T3D.gro -p prod_structures/APO/APO_6T3D.top -water tip3p -ff amber99sb-ildn -ignh

# TODO Check HIS states - Not accurate, taken from pdb2gmx -nointer
# TODO Check Protonation states - not complete in some cases
# TODO Check partial charges - fabricated

In [ ]:
KCX_angle_types = """"
[ angletypes ]
;  i    j    k  func       th0       cth
HP  CT  N            1   108.880    514.630 ; Added by Alexi 31/07/2023
N   C   O2           1   123.050    952.280 ; Added by Alexi 31/07/2023
"""
# need to add this to the topology file and the forcefield file 

In [ ]:

input_path = os.path.join(settings.structures_output, test_pdb)
output_path = os.path.join(settings.structures_output, test_pdb.replace(".pdb", ".gro"))
topo_path = os.path.join(settings.structures_output, test_pdb.replace(".pdb", ".top"))

subprocess.run(["gmx", "pdb2gmx", 
                "-f", input_path, 
                "-o", output_path, 
                "-p", topo_path, "-water", "tip3p", 
                "-ff", "amber99sb-ildn",
                "-ignh",], 
                check=True)


In [ ]:
# Prep simulations - set box size, solvate and ions

# use GMXAPI


